In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df= pd.read_csv('credit-card-default.csv')
df.head()



,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,defaulted
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   ID         30000 non-null  int64
 1   LIMIT_BAL  30000 non-null  int64
 2   SEX        30000 non-null  int64
 3   EDUCATION  30000 non-null  int64
 4   MARRIAGE   30000 non-null  int64
 5   AGE        30000 non-null  int64
 6   PAY_0      30000 non-null  int64
 7   PAY_2      30000 non-null  int64
 8   PAY_3      30000 non-null  int64
 9   PAY_4      30000 non-null  int64
 10  PAY_5      30000 non-null  int64
 11  PAY_6      30000 non-null  int64
 12  BILL_AMT1  30000 non-null  int64
 13  BILL_AMT2  30000 non-null  int64
 14  BILL_AMT3  30000 non-null  int64
 15  BILL_AMT4  30000 non-null  int64
 16  BILL_AMT5  30000 non-null  int64
 17  BILL_AMT6  30000 non-null  int64
 18  PAY_AMT1   30000 non-null  int64
 19  PAY_AMT2   30000 non-null  int64
 20  PAY_AMT3   30000 non-null  int64
 21  PAY_AMT4   3

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop('defaulted', axis=1)
y = df['defaulted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)
print(f"Train Data: {X_train.shape},{y_train.shape}")
print(f"Test Data: {X_test.shape},{y_test.shape}")

Train Data: (21000, 24),(21000,)
Test Data: (9000, 24),(9000,)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

In [ ]:
pipeline = make_pipeline(StandardScaler(),DecisionTreeClassifier(random_state=1))

pipeline.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=1))])

In [ ]:
print(f"Model test Score: {pipeline.score(X_test, y_test)}")
print(f"Model train Score: {pipeline.score(X_train, y_train)}")

Model test Score: 0.7275555555555555
Model train Score: 1.0


The model comes up with the above scores. Note that the model tends to overfit the data as the test score is 0.72 and training score is 1.00.

In [ ]:
from sklearn.ensemble import BaggingClassifier
bgclassifier = BaggingClassifier(base_estimator=pipeline, random_state=1)
bgclassifier.fit(X_train,y_train)

print(f"Model test Score: {pipeline.score(X_test, y_test)}")
print(f"Model train Score: {pipeline.score(X_train, y_train)}")

Model test Score: 0.7275555555555555
Model train Score: 1.0


In [ ]:
from sklearn.model_selection import GridSearchCV
'''A dictionary with base_estimator__max_depth from 1 to 20 with step size as 5
A max_samples with [0.05, 0.1, 0.2, 0.5] and max_features form 1 to 20 with step size as 5'''
param_grid = {
    'estimator__max_depth' : [1,5,10,15,20],
    'max_samples': [0.05, 0.1, 0.2, 0.5],
    'max_features': [1, 5, 10, 15, 20]
}
gs = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(),random_state=1), 
                   param_grid,scoring='accuracy')
gs.fit(X_train, y_train)

GridSearchCV(estimator=BaggingClassifier(estimator=DecisionTreeClassifier(),
                                         random_state=1),
             param_grid={'estimator__max_depth': [1, 5, 10, 15, 20],
                         'max_features': [1, 5, 10, 15, 20],
                         'max_samples': [0.05, 0.1, 0.2, 0.5]},
             scoring='accuracy')

In [ ]:
print("Optimal Hyperparameter Combination:", gs.best_params_)
print("Mean cross-validated training accuracy score:", gs.best_score_)

Optimal Hyperparameter Combination: {'estimator__max_depth': 5, 'max_features': 20, 'max_samples': 0.5}
Mean cross-validated training accuracy score: 0.8177619047619048


In [ ]:
bgclassifier = BaggingClassifier(DecisionTreeClassifier(max_depth = 5),max_features=20,max_samples=0.5,random_state=1)
bgclassifier.fit(X_train, y_train)

print(f"Model test Score: {bgclassifier.score(X_test, y_test)}")
print(f"Model train Score: {bgclassifier.score(X_train, y_train)}")

Model test Score: 0.8237777777777778
Model train Score: 0.8231428571428572
